In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import tensorflow as tf
import gensim

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
fake = pd.read_csv('/content/Fake.csv')

real = pd.read_csv('/content/True.csv')
real.head()

real.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [8]:
for index, row in enumerate(real.text.values):
  # record = row.split(" -", maxsplit = 1)
  # if(len(record) != 1 and len(record[0]) < 260):
  #   real.text[index] = record[1:]
  if(str(row).strip() == '' or len(row) > 800):
    real = real.drop(index)

for index, row in enumerate(fake.text.values):
  # record = row.split(" -", maxsplit = 1)
  # if(len(record) != 1 and len(record[0]) < 260):
  #   fake.text[index] = record[1]
  if(str(row).strip() == '' or len(row) > 800):
    fake = fake.drop(index)


In [9]:
real['class'] = 1
fake['class'] = 0

real["text"] = real["title"] + real["text"]
fake["text"] = fake["title"] + fake["text"]
real = real.drop(['subject', "date", "title"], axis = 1)
fake = fake.drop(['subject', "date", "title"], axis = 1)

data = real.append(fake, ignore_index=True)

data.head()

,text,class
0,Trump on Twitter (Dec 28) - Global WarmingThe ...,1
1,Alabama official to certify Senator-elect Jone...,1
2,Factbox: Trump on Twitter (Dec 28) - Vanity Fa...,1
3,Mexico to review need for tax changes after U....,1
4,House panel asks Trump ex-top aide Bannon to t...,1


In [10]:

y = data["class"].values
#Converting X to format acceptable by gensim, removing annd punctuation stopwords in the process
x = []
stop_words = set(nltk.corpus.stopwords.words("english"))
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in data["text"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
        tmp.extend(filtered_words)
    x.append(tmp)


In [15]:
w2v_model = gensim.models.Word2Vec(sentences = x, size = 100, window = 5, min_count = 1)

w2v_model.wv.most_similar("trump")

[('tpa', 0.9212424755096436),
 ('threatsthese', 0.9019190073013306),
 ('trumpsarah', 0.8872480988502502),
 ('president', 0.8765723705291748),
 ('ditches', 0.8631701469421387),
 ('pencebrussels', 0.8552950620651245),
 ('revoked', 0.8516873121261597),
 ('show21st', 0.8457914590835571),
 ('pundithis', 0.8457082509994507),
 ('aiello', 0.839814305305481)]

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)
 
x = pad_sequences(x, maxlen = 800)

In [21]:
from sklearn.model_selection import train_test_split

layers = tf.keras.layers
model = tf.keras.models.Sequential()
model.add(layers.Embedding(len(tokenizer.word_index)+1, output_dim=100, weights=[np.append(w2v_model.wv.syn0,(np.zeros((1, 100))), axis = 0)], input_length=800, trainable=False))
model.add(layers.LSTM(units=128))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

xtrain, xtest, ytrain, ytest = train_test_split(x, y) 

model.fit(xtrain, ytrain, epochs = 3, validation_split=0.1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """


Epoch 1/3
158/158 [==============================] - 120s 746ms/step - loss: 0.5210 - acc: 0.7479 - val_loss: 0.5863 - val_acc: 0.6441
Epoch 2/3
158/158 [==============================] - 117s 742ms/step - loss: 0.4873 - acc: 0.7678 - val_loss: 0.1428 - val_acc: 0.9573
Epoch 3/3
158/158 [==============================] - 117s 742ms/step - loss: 0.0642 - acc: 0.9830 - val_loss: 0.0155 - val_acc: 0.9982


In [23]:
from sklearn.metrics import accuracy_score
ypred = (model.predict(xtest) >= 0.5).astype("int")
accuracy_score(ytest, ypred)


1.0